In [14]:
import pandas as pd
import numpy as np
import math
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from AucModule import calcAUC

In [15]:
data = pd.read_csv('data/train_new.csv')

In [16]:
data.dropna(thresh = 60, inplace = True)

In [17]:
data.dropna(subset=['Y'], inplace = True)

In [18]:
# fillna with means
for i in range(1, 73):
    data_col = data['X%s'%i].fillna(data['X%s'%i].mean())
    data['X%s'%i] = data_col

In [19]:
woe = {}
woel = []
good_t = sum(data['Y']==1)
bad_t = sum(data['Y']==0)

In [20]:
for i, v in data.X13.items():
    good = 0
    bad = 0
    if woe.get(v) != None:
        woel.append(woe[v])
    else:
        for j, vj in data.X13.items():
            if vj == v:
                if data.Y[j] == 1:
                    good += 1
                else:
                    bad += 1
        woel.append(math.log((good/good_t+0.1)/(bad/bad_t+0.1)))
        woe[v] = math.log((good/good_t+0.1)/(bad/bad_t+0.1))

In [21]:
data["woel"] = woel

In [22]:
def add_cross_feature(data,feature1,feature2):
    comb_index = data[[feature1,feature2]].drop_duplicates()
    comb_index[feature1+'_'+feature2]=data[feature1]+data[feature2]
    data = pd.merge(data, comb_index, 'left', on=[feature1,feature2])
    return data

In [23]:
data = add_cross_feature(data,'X23','X34')
data = add_cross_feature(data,'X36','X37')

In [24]:
# split test data and train data
x = pd.concat([data.iloc[:, :72],data.iloc[:, [74]]], axis = 1)
y = data.iloc[:, [72]]           # here is an error (73->72)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [25]:
# z-score standard
x_train_arr = np.array(x_train)
x_test_arr = np.array(x_test)
s1 = StandardScaler()
s2 = StandardScaler()
s1.fit(x_train_arr)
x_train_stand = s1.transform(x_train_arr)
s2.fit(x_test_arr)
x_test_stand = s2.transform(x_test_arr)

In [26]:
start_time = time.time()
clf = LogisticRegression(random_state=0).fit(x_train_stand, y_train.values.ravel())
end_time = time.time()
prey = clf.predict(x_test_stand)
print("time: %s" % (end_time-start_time))

time: 0.5023133754730225


E:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [41]:
print("auc_score: %s" % calcAUC(y_test.Y,prey))

auc_score: 0.6148069111308395


In [123]:
name = ["id", "predict"]
list = []
for i in range(prey.shape[0]):
    list.append([x_test.index[i], prey[i]])
pred = pd.DataFrame(columns=name, data=list)

In [126]:
pred.to_csv("./result/LR")